In [1]:
import pandas as pd
import numpy as np

In [2]:
demographic_data = pd.read_csv('../data/raw/s3/demographics_data.csv', parse_dates=['reg_date'], infer_datetime_format=True )
demographic_data.uid = demographic_data.uid.astype('int')
print(demographic_data.shape)
demographic_data.head()

(10000, 6)


,uid,reg_date,device,gender,country,age
0,52774929,2018-03-07,and,F,FRA,27
1,84341593,2017-09-22,iOS,F,TUR,22
2,41201055,2017-11-24,and,F,USA,20
3,68477880,2016-12-08,and,F,BRA,18
4,42235969,2017-09-30,and,F,USA,41


In [3]:
paywall_views = pd.read_csv('../data/raw/s3/paywall_views.csv', parse_dates=['date'], infer_datetime_format=True)
paywall_views.head()

,uid,date,purchase,sku,price
0,32209877,2016-12-04 14:20:49,0,NaN,NaN
1,32209877,2016-12-05 22:17:12,0,NaN,NaN
2,32209877,2016-12-09 17:56:09,0,NaN,NaN
3,32209877,2016-12-13 03:07:08,0,NaN,NaN
4,32209877,2016-12-14 17:19:29,0,NaN,NaN


In [4]:
purchase_data = demographic_data.merge(paywall_views, how='left', on='uid')
purchase_data.head()

,uid,reg_date,device,gender,country,age,date,purchase,sku,price
0,52774929,2018-03-07,and,F,FRA,27,2018-03-10 22:25:17,0.0,NaN,NaN
1,52774929,2018-03-07,and,F,FRA,27,2018-03-11 09:06:00,0.0,NaN,NaN
2,52774929,2018-03-07,and,F,FRA,27,2018-03-12 20:06:41,0.0,NaN,NaN
3,84341593,2017-09-22,iOS,F,TUR,22,NaT,NaN,NaN,NaN
4,41201055,2017-11-24,and,F,USA,20,NaT,NaN,NaN,NaN


In [5]:
total_purchases = purchase_data.groupby(by='uid', as_index=False).purchase.sum()
total_purchases.mean()

uid         5.601861e+07
purchase    3.465000e-01
dtype: float64

In [6]:
total_purchases.purchase.min()

0.0

In [7]:
total_purchases.purchase.max()

17.0

In [8]:
total_purchases = purchase_data.groupby(by=['date', 'uid'], as_index=False).purchase.sum()
print(total_purchases.purchase.mean())
print(total_purchases.purchase.min())
print(total_purchases.purchase.max())

0.03468607351645712
0.0
1.0


In [9]:
df1 = pd.read_csv('../data/raw/s3/demographics_data.csv', parse_dates=['reg_date'], infer_datetime_format=True )
df2 = pd.read_csv('../data/raw/s3/purchase_data.csv', parse_dates=['date'], infer_datetime_format=True)
df2.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 128412 entries, 0 to 128411
Data columns (total 4 columns):
 #   Column  Non-Null Count   Dtype         
---  ------  --------------   -----         
 0   date    128412 non-null  datetime64[ns]
 1   uid     128412 non-null  float64       
 2   sku     128412 non-null  object        
 3   price   128412 non-null  int64         
dtypes: datetime64[ns](1), float64(1), int64(1), object(1)
memory usage: 3.9+ MB


In [10]:
purchase_data = df2.merge(df1, how='inner', on='uid')
purchase_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 128412 entries, 0 to 128411
Data columns (total 9 columns):
 #   Column    Non-Null Count   Dtype         
---  ------    --------------   -----         
 0   date      128412 non-null  datetime64[ns]
 1   uid       128412 non-null  float64       
 2   sku       128412 non-null  object        
 3   price     128412 non-null  int64         
 4   reg_date  128412 non-null  datetime64[ns]
 5   device    128412 non-null  object        
 6   gender    128412 non-null  object        
 7   country   128412 non-null  object        
 8   age       128412 non-null  int64         
dtypes: datetime64[ns](2), float64(1), int64(2), object(4)
memory usage: 9.8+ MB


In [11]:
# Extract the 'day'; value from the timestamp
purchase_data.date = purchase_data.date.dt.floor('d')

# Replace the NaN price values with 0 
purchase_data.price = np.where(np.isnan(purchase_data.price), 0, purchase_data.price)

# Aggregate the data by 'uid' & 'date'
purchase_data_agg = purchase_data.groupby(by=['uid', 'date'], as_index=False)
revenue_user_day = purchase_data_agg.sum()

# Calculate the final average
revenue_user_day = revenue_user_day.price.mean()
print(revenue_user_day)

407.33800579385104


In [12]:
df1 = pd.read_csv('../data/raw/s3/demographics_data.csv', parse_dates=['reg_date'], infer_datetime_format=True )
df2 = pd.read_csv('../data/raw/s3/paywall_views.csv', parse_dates=['date'], infer_datetime_format=True)
purchase_data = df1.merge(df2, how='left', on='uid')
purchase_data.head()

,uid,reg_date,device,gender,country,age,date,purchase,sku,price
0,52774929,2018-03-07,and,F,FRA,27,2018-03-10 22:25:17,0.0,NaN,NaN
1,52774929,2018-03-07,and,F,FRA,27,2018-03-11 09:06:00,0.0,NaN,NaN
2,52774929,2018-03-07,and,F,FRA,27,2018-03-12 20:06:41,0.0,NaN,NaN
3,84341593,2017-09-22,iOS,F,TUR,22,NaT,NaN,NaN,NaN
4,41201055,2017-11-24,and,F,USA,20,NaT,NaN,NaN,NaN


In [13]:
total_revenue = purchase_data.groupby(by=['uid'], as_index=False).price.sum()
total_revenue.price = np.where(np.isnan(total_revenue.price), 0 , total_revenue.price)
average_revenue = total_revenue.price.mean()
average_revenue

1.7777349999999998

In [14]:
revenue_variation = total_revenue.price.std()
revenue_variation

7.701463828018958

In [15]:
revenue_variation / average_revenue

4.3321776462852775

In [16]:
purchase_data = df1.merge(df2, how='inner', on='uid')
conversion_rate = (sum(purchase_data.purchase) / purchase_data.purchase.count())
conversion_rate

0.03468607351645712

In [17]:
purchase_data.date = purchase_data.date.dt.floor('d')

# Group and aggregate our combined data set 
daily_purchase_data = purchase_data.groupby(by=['date'], as_index=False)
daily_purchase_data = daily_purchase_data.agg({'purchase': ['sum', 'count']})

# Find the mean of each field and then multiply by 1000 to scale the result
daily_purchases = daily_purchase_data.purchase['sum'].mean()
daily_paywall_views = daily_purchase_data.purchase['count'].mean()
daily_purchases = daily_purchases * 1000
daily_paywall_views = daily_paywall_views * 1000

print(daily_purchases)
print(daily_paywall_views)

3181.8181818181815
91731.86409550045


In [18]:
small_sensitivity = 0.1 

# Find the conversion rate when increased by the percentage of the sensitivity above
small_conversion_rate = conversion_rate * (1 + small_sensitivity) 

# Apply the new conversion rate to find how many more users per day that translates to
small_purchasers = daily_paywall_views * small_conversion_rate

# Subtract the initial daily_purcahsers number from this new value to see the lift
purchaser_lift = small_purchasers - daily_purchases

print(small_conversion_rate)
print(small_purchasers)
print(purchaser_lift)

0.038154680868102836
3500.0000000000005
318.1818181818189


In [19]:
medium_sensitivity = 0.2

# Find the conversion rate when increased by the percentage of the sensitivity above
medium_conversion_rate = conversion_rate * (1 + medium_sensitivity) 

# Apply the new conversion rate to find how many more users per day that translates to
medium_purchasers = daily_paywall_views * medium_conversion_rate

# Subtract the initial daily_purcahsers number from this new value to see the lift
purchaser_lift = medium_purchasers - daily_purchases

print(medium_conversion_rate)
print(medium_purchasers)
print(purchaser_lift)

0.04162328821974854
3818.181818181818
636.3636363636365


In [20]:
large_sensitivity = 0.5

# Find the conversion rate lift with the sensitivity above
large_conversion_rate = conversion_rate * (1 + large_sensitivity)

# Find how many more users per day that translates to
large_purchasers = daily_paywall_views * large_conversion_rate
purchaser_lift = large_purchasers - daily_purchases

print(large_conversion_rate)
print(large_purchasers)
print(purchaser_lift)

0.05202911027468568
4772.727272727273
1590.9090909090914


In [21]:
# Find the number of paywall views 
n = purchase_data.purchase.count()

# Calculate the quantitiy "v"
v = conversion_rate * (1 - conversion_rate) 

# Calculate the variance and standard error of the estimate
var = v / n 
se = var**0.5

print(var)
print(se)

3.351780834114284e-07
0.0005789456653360731


In [22]:
from scipy import stats
def get_power(n, p1, p2, cl):
    alpha = 1 - cl
    
    qu = stats.norm.ppf(1 - alpha/2) # 0.025 / 0.025
    
    diff = abs(p2 - p1)
    bp = (p1 + p2) / 2
    
    v1 = p1 * (1 - p1)
    v2 = p2 * (1 - p2)
    
    bv = bp * (1 - bp)
    
    power_part_one = stats.norm.cdf((n**0.5 * diff - qu * (2 * bv)**0.5)/ (v1 + v2)**0.5)
    power_part_two = 1 - stats.norm.cdf((n**0.5 * diff + qu * (2 * bv)**0.5)/ (v1 + v2)**0.5)
    
    power = power_part_one + power_part_two
    return(power)

In [23]:
p1 = 0.1
p2 = 0.12
cl = 0.95
n1 = 1000
# Look at the impact of sample size increase on power
n_param_one = get_power(n=1000, p1=p1, p2=p2, cl=cl)
n_param_two = get_power(n=2000, p1=p1, p2=p2, cl=cl)

# Look at the impact of confidence level increase on power
alpha_param_one = get_power(n=n1, p1=p1, p2=p2, cl=0.8)
alpha_param_two = get_power(n=n1, p1=p1, p2=p2, cl=0.95)
    
# Compare the ratios
print(n_param_two / n_param_one)
print(alpha_param_one / alpha_param_two)

1.7596440001351992
1.8857367092232278


In [24]:
stats.norm.ppf(1-0.05/2)

1.959963984540054

In [25]:
def get_sample_size(power, p1, p2, cl, max_n=1000000):
    n = 1 
    while n <= max_n:
        tmp_power = get_power(n, p1, p2, cl)

        if tmp_power >= power: 
            return n 
        else: 
            n = n + 100

    return "Increase Max N Value"

In [37]:
                            
# Find the conversion rate
conversion_rate = (sum(purchase_data.purchase) / purchase_data.purchase.count())
print(conversion_rate)            
# Desired Power: 0.8
# CL: 0.90
# Percent Lift: 0.1
p2 = conversion_rate * (1 + 0.1)
sample_size = get_sample_size(0.8, conversion_rate, p2, 0.90)
print(sample_size, p2)

0.03468607351645712
36101 0.038154680868102836


In [39]:
sample_size = get_sample_size(0.95, 0.03, 0.03, 0.90)
print(sample_size)

Increase Max N Value


In [36]:
# Desired Power: 0.95
# CL: 0.90
# Percent Lift: 0.1
p2 = conversion_rate * (1 + 0.1)
sample_size = get_sample_size(0.95, conversion_rate, p2, 0.90)
print(sample_size)

63201
